In [ ]:
import math
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from botorch.models.gpytorch import GPyTorchModel
from gpytorch.models import ExactGP
from gpytorch.distributions import MultivariateNormal
from gpytorch.means import ConstantMean
from gpytorch.kernels import RBFKernel, ScaleKernel, SpectralMixtureKernel, LinearKernel, PeriodicKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.priors import GammaPrior
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader,TensorDataset

from botorch import fit_gpytorch_model
from botorch.exceptions import BadInitialCandidatesWarning

In [98]:
#  Define the model

class ExactGPModel(ExactGP,GPyTorchModel):
    
    _num_outputs = 1
    
    def __init__(self,train_x,train_y):
        super().__init__(train_x,train_y.squeeze(-1),GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(SpectralMixtureKernel(5,1) + RBFKernel(train_x.shape[-1]) 
                                       
                                       )
        self.to(train_x)
        
    def forward(self,x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        
        return MultivariateNormal(mean_x,covar_x)

In [47]:
#  Initialize the model
def Initialize_model(train_x,train_y,state_dict=None):
    model = ExactGPModel(train_x,train_y)
    mll = ExactMarginalLogLikelihood(model.likelihood,model)
    if state_dict is not None:
        model.load_state_dict(state_dict)
    return model,mll

In [48]:
def next_query_point(model,rem_x):
    model.eval()
    model.likelihood.eval()
    
    var = model.likelihood(model(rem_x)).variance
    ind = torch.argmax(var)
    
    return ind
    

In [122]:
import warnings 

warnings.filterwarnings('ignore',category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore',category=RuntimeWarning)

def fit(x,y,verbose=True):
    np.random.seed(42)
    # Training data for working model--> need replace with out original data
    l = int(torch.floor(torch.tensor(0.8*114)).item())
    scaler = StandardScaler()
    train_x = x[5:l]
    train_y = y[5:l]

    train_x = torch.Tensor(scaler.fit_transform(train_x))

    test_x = torch.Tensor(scaler.transform(x[l:]))
    test_y = y[l:]

    train_dataset = TensorDataset(train_x,train_y)

    train_loader = DataLoader(train_dataset,batch_size=10,shuffle=True)

    # Initialize the model
    train_x_al = train_x[:5]
    train_y_al = train_y[:5]
    model_al,mll_al = Initialize_model(train_x_al,train_y_al)
    model_al.state_dict()
    model_al.train()

    for b,(rem_x,rem_y) in enumerate(train_loader):
    
        print(f"Batch {b+1:>3}")
        N_trails = len(rem_x)

    
        for i in range(N_trails):
        
            ind = next_query_point(model_al,rem_x)
            new_x = rem_x[ind]
            new_y = rem_y[ind]
        
    
            rem_x = torch.cat((rem_x[:ind],rem_x[ind+1:]))
            rem_y = torch.cat((rem_y[:ind],rem_y[ind+1:]))
        
        
            # update the training points 
            train_x_al = torch.cat((train_x_al,new_x.reshape(1,1)))
            train_y_al = torch.cat((train_y_al,new_y.reshape(1,1)))
        
            # reinitiate the model
            model_al,mll_al = Initialize_model(train_x_al,train_y_al,model_al.state_dict())
        
            fit_gpytorch_model(mll_al)
        
            
            optimizer = torch.optim.Adam(model_al.parameters(),lr=0.01)
        
            epochs = 10
        
            model_al.train()
        
            for epoch in range(epochs):
            
            
                # predicting on forward pass
                output = model_al(train_x_al)
            
                # compute negative marginal log likelihood
            
                loss = -mll_al(output,model_al.train_targets)
            
                optimizer.zero_grad()
            
                loss.backward()
            
                if (epoch + 1)%10 == 0 and verbose:
                    print(
                        f"Epoch {epoch+1:>3}/{epochs} - Loss:{loss.item():>4.3f},lengthscale:{model_al.covar_module.base_kernel.lengthscale.item()}")
            
                optimizer.step()
    model_al.eval()
    model_al.likelihood.eval()
    
    predictions = model_al.likelihood(model_al(test_x))
    
    return test_y, predictions
        

Batch   1/1
Epoch  10/10 - Loss:-0.440
Epoch  10/10 - Loss:-0.249
Epoch  10/10 - Loss:-0.036
Epoch  10/10 - Loss:-0.088
Epoch  10/10 - Loss:-0.116
Epoch  10/10 - Loss:-0.325
Epoch  10/10 - Loss:-0.435
Epoch  10/10 - Loss:-0.632
Epoch  10/10 - Loss:-0.245
Epoch  10/10 - Loss:-0.064
Epoch  10/10 - Loss:0.160
Epoch  10/10 - Loss:0.083
Epoch  10/10 - Loss:0.199
Epoch  10/10 - Loss:0.209
Epoch  10/10 - Loss:0.177
Epoch  10/10 - Loss:0.160
Epoch  10/10 - Loss:0.105
Epoch  10/10 - Loss:0.425
Epoch  10/10 - Loss:0.407
Epoch  10/10 - Loss:0.385
Epoch  10/10 - Loss:0.489
Epoch  10/10 - Loss:0.478
Epoch  10/10 - Loss:0.462
Epoch  10/10 - Loss:0.462
Epoch  10/10 - Loss:0.660
Epoch  10/10 - Loss:0.641
Epoch  10/10 - Loss:0.624
Epoch  10/10 - Loss:0.647
Epoch  10/10 - Loss:0.630
Epoch  10/10 - Loss:0.619
Epoch  10/10 - Loss:0.604
Epoch  10/10 - Loss:0.626
Epoch  10/10 - Loss:0.636
Epoch  10/10 - Loss:0.622
Epoch  10/10 - Loss:0.610


In [ ]:
cw_avg = pd.read_csv('cosine_weighted_average.csv')
total_x = torch.linspace(1901,2014,114).reshape(-1,1).double()
total_y = torch.tensor(cw_avg.values).reshape(-1,1).double()
actual_y_nor, predicted_y_nor = fit(total_x,total_y,verbose=False)

In [ ]:
abs(actual_y_nor-predicted_y_nor.mean).mean()

In [ ]:
(abs(actual_y_nor-predicted_y_nor.mean)/actual_y_nor).mean()

In [ ]:
op = [25,40,50,75]
dy = [10,20,30,40,50]

for i in op:
    print('optimal_locations:',i)
    df = pd.read_table('absolute_error_op_'+str(i)+'_rs.dat',sep='\s+',
                      header=None)
    df.columns = [str(j) for j in range(5)]
    for k in range(5):
        df_x = pd.read_csv('OA_evaluated_using_optimal_subset_for_duration_'+str(dy[k])+'.csv')
        print('duration of years:',dy[k])
        total_x = df_x[str(int(20*i))+' locations'].values.reshape(-1,1)
        assert(total_x.shape==(114,1))
        total_y = torch.tensor(df[str(k)].values).reshape(-1,1).double()
        assert(total_y.shape==(114,1))
        actual_y, predicted_y = fit(total_x,total_y,verbose=False)
        assert(len(actual_y)==23)
        assert(len(predicted_y.mean)==23)
        con_cat = torch.cat((predicted_y.mean.reshape((-1,1)),actual_y.reshape((-1,1))),dim=1)
        com_df = pd.DataFrame(con_cat.detach().numpy(),index=[str(i) for i in range(1992,2015,1)],
                     columns=['observed_y','actual_y'])
        com_df.to_csv('predicted_SMK_'+str(dy[k])+'_years_'+str(i)+'_op_rs_with_input_cbar.dat')

In [ ]:
dict_abs_err = {'10 years':[],'20 years':[],'30 years':[],'40 years':[],'50 years':[]}
dict_rel_err = {'10 years':[],'20 years':[],'30 years':[],'40 years':[],'50 years':[]}

for j in dy:
    for i in op:
        inputs = pd.read_csv('OA_evaluated_using_optimal_subset_for_duration_'+str(j)+'.csv')[str(int(20*i))+' locations']
        error = pd.read_csv('predicted_SMK_'+str(j)+'_years_'+str(i)+'_op_rs_with_input_cbar.dat')
        corrected = inputs.values.reshape(-1,1)[-23:,0:] + error['observed_y'].values.reshape(-1,1)
        true_values = cw_avg['cw_avg'].iloc[-23:].values.reshape(-1,1)
        absolute_error = abs(true_values-corrected)
        assert(len(absolute_error)==len(true_values))
        relative_error = absolute_error/true_values
        assert(len(relative_error)==len(true_values))
        dict_abs_err[str(j)+' years'].append(np.mean(absolute_error))
        dict_rel_err[str(j)+' years'].append(np.mean(relative_error))
#         print(f'''mean absolute error for {i*20} optimal 
#         locations considering {j} years of data is {np.mean(absolute_error)}''')
#         print(f'''mean relative error for {i*20} optimal 
#         locations considering {j} years of data is {np.mean(relative_error)}''')
                         